In [44]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm

file_exts = ['.jpg', '.png', '.cr2']

images_paths = [
    # '/home/jack/Mounts/RemoteS3/kona_coffee_images/samples/all',
    # '/home/jack/Mounts/RemoteS3/kona_coffee_images/raw/02-12-2023/raws',
    '/home/jack/Mounts/RemoteS3/kona_coffee_images/raw/02-12-2023/phone'
]

fun_path = '/home/jack/Mounts/DiskOne/kona_coffee/fun'
splits_path = '/home/jack/Mounts/DiskOne/kona_coffee/splits'
box_size = 400

os.makedirs(splits_path, exist_ok=True)

In [45]:
def recursive_list(dir_path, any=False):
    file_list = []
    for root, directories, filenames in os.walk(dir_path):
        for filename in filenames:
            file_path = os.path.join(root, filename)
            if os.path.isfile(file_path):
                if any:
                    file_list.append(file_path)
                elif os.path.splitext(file_path)[1].lower() in file_exts:
                    file_list.append(file_path)
    return file_list

In [46]:
images = [image for path in images_paths for image in recursive_list(path)]
fun_images = os.listdir(fun_path)

for image_path in tqdm(images):
    extention = os.path.splitext(image_path)[1].lower()
    image_name = os.path.basename(image_path)
    
    if image_name in fun_images:
        continue
    
    image = None
    
    if extention == '.jpg' or extention == '.png':
        image = Image.open(image_path)  
    
    elif extention == '.cr2':
        image = Image.open(image_path)
        image = image.convert('RGB')

    splits = []
    width, height = image.size

    for i in range(0, width, box_size):
        for j in range(0, height, box_size):
            box = (i, j, i+box_size, j+box_size)
            new_name = os.path.splitext(image_name)[0] + '_' + str(i) + '_' + str(j) + '.png'
            new_path = os.path.join(splits_path, new_name)

            if os.path.exists(new_path):
                continue

            split = image.crop(box)            
            split.save(new_path)

100%|██████████| 266/266 [19:49<00:00,  4.47s/it]


In [47]:
len(os.listdir(splits_path))

35370